# 20 特征数据与模型参数的处理

## 20.1 机器学习可能面临的若干问题

### 20.1.1 数据的问题


- 数据未必是现成的 `属性/目标` 配对
    - 可能更复杂(不同构)
        - 图片
        - 缺陷
    - 也可能根本不成(数据)结构
        - 文本
        - 音视频


- 基本技能 —— `特征提取(Feature Extraction)`
    - 提取（提炼） `scikit-learn` 能用的特征
    - ......

### 20.1.2 特征的问题


- 初始数据集所包含的
    - 未必都是有用的特征
    - 甚至有些极低信息量(无用、无关、冗余......)的数据特征


- 重要技能 —— `特征选择(Feature Selection)`
    - 选择最恰当的特征集

### 20.1.3 参数的问题


- 前述 `有/无 监督学习` 表明
    - 机器学习算法需要 `设置/调整 参数`


- 关键技能 —— `模型选择(Model Selection)`
    - 找到最理想的参数

## 20.2 特征提取 (Feature Extraction)

### 20.2.1 机器学习流程与数据准备困难


- 机器学习的任务流程
    - 数据准备
        - 多样本
        - 多特征 + (多)目标
        - 数据预处理
    - 通过训练数据的学习，获得经验
    - 对于将来未知的样本特征数据，预测其目标值


- 数据准备的困难
    - 源数据可能不是适宜的数据格式
        - 20 新闻组
        - 非统一格式照片
        - 其它格式
    - 有必要
        - 提取潜在的有用特征
        - 并将其转换成可用的格式

### 20.2.2 特征提取的实质


- 提取潜在的有用特征称为
    - 特征提取 或 特征工程

#### (1) 步骤


- 处理源数据
    - 整理、归类、存档等


- 提取样本数据
    - 通常的格式：特征值/目标值
    - 值的类型：整数、浮点数、字符串、分类值等
        - 一律要转换成数值

#### (2) 方法 —— 无定规，（强）依赖于数据的表现形式


- 贴标签(特征或目标) —— 如 `Titanic` 号生还预测、鸢尾花分类等
- 提取数据(特征或目标) —— 如 `20` 新闻组 `tfidf` 等
- 人工生成(特征或目标) —— 如头像识别判断是否戴眼镜
- 使用各种工具
- 其它 ......

### 20.2.3 转换特征


- Scikit-Learn 对样本数据的要求
    - 数据集 —— 样本数据组成的集合
    - 样本 = 多个特征值 + 目标值
    - 值：必须是浮点或整数值


- 现状
    - 得到这样的数据并不总是轻松


- 办法
    - 专门编写转换程序
    - 利用软件模块，如 `pandas` 等

### 20.2.4 回顾 Titanic 乘客生还预测问题


- 模块准备

In [ ]:
%pylab inline
import IPython

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import sklearn as sk
import pandas as pd

print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('matplotlib version:', matplotlib.__version__)

print ('scikit-learn version:', sk.__version__)
print ('pandas version:', pd.__version__)

- 加载数据

In [ ]:
titanic = pd.read_csv('data/titanic.csv')
titanic

- 观察数据
    - 共有 1313 行、11 列

In [ ]:
titanic.columns

- 数据帧的每一列
    - 对应一个特征(或目标)
    - 有些是值
    - 有些是分类值 —— 如 `embarked` 列的 `Southampton`、`Cherbourg` 等

In [ ]:
titanic.head()[['pclass', 'survived', 'age', 'embarked', 'boat', 'sex']]

- 遇到困难
    - `scikit-learn` 希望的特征是实数值
    - 需要做转换


- 曾经用过工具转换数据
    - `LabelEncoder`
    - `OneHotEncoder`


- 新的转换工具
    - `DictVectorizer`

### 20.2.5 转换特征工具 —— `DictVectorizer`


- 字典向量化器 —— 特征提取工具
- 利用原特征名称，组合新特征
- 采用 `0/1` 方式进行量化

#### (1) 导入 feature_extraction 模块

In [ ]:
from sklearn import feature_extraction

#### (2) 定义函数


- 函数 `one_hot_dataframe` 调用方式

```python
>>> new_data_frame, vec_data_frame = one_hot_dataframe(data, cols, replace=False)
```


- 功能 —— 转换特征成新数据帧


- 参数
    - `data` 数据帧
    - `cols` 指定若干列
    - `replace` 是否替换 `data`

In [ ]:
def one_hot_dataframe(data, cols, replace=False):
    """ 
    取数据帧和有待解码的若干列
    返回：数据data、向量化数据，以及拟合的向量化器
    """
    vec = feature_extraction.DictVectorizer()                   # 生成 DictVectorizer 对象
    mkdict = lambda row: dict((col, row[col]) for col in cols) # 映射：列标识 -> 行的对应元素
    
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray()) # 重构数据帧
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)

    return (data, vecData)

#### (3) 转换 `titanic` 数据帧的三列 `pclass`、`embarked` 和 `sex`

In [ ]:
titanic, titanic_n = one_hot_dataframe(titanic, ['pclass', 'embarked', 'sex'], replace=True)
titanic

In [ ]:
titanic.describe()

#### (4) 观察新数据帧


- 由 `pclass` 特征 生出 三个新特征
    - `pclass=1st`
    - `pclass=2nd`
    - `pclass=3rd`
- 由 `embarked` 特征 生出 三个新特征
    - `embarked=Cherbourg`
    - `embarked=Queenstown`
    - `embarked=Southampton`
- 由 `sex` 特征 生出 两个新特征
    - `sex=female`
    - `sex=male`

#### (5) 特征的存与灭


- 失踪的特征
    - `embarked` 还存在
    - `pclass` 和 `sex` 不见了


- 原因 —— 原 `embarked` 特征中有丢失的值 `NaN`


- 注意
    - 原 `embarked` 值存在时，新 `embarked` 值为 0
    - 原 `embarked` 值为 `NaN` 时，新 `embarked` 值仍为 `NaN`

#### (6) 小心遭遇魔咒 —— 再解码一组特征

In [ ]:
titanic, titanic_n = one_hot_dataframe(titanic, ['home.dest', 'room', 'ticket', 'boat'], replace=True)
titanic

- 解码引起特征数急剧增长，达到 581 个特征

In [ ]:
titanic.columns

#### (7) 处理丢失的值


- 许多算法都不能接受 `NaN`
- 可以使用 `Pandas` 的 `fillna` 方法

In [ ]:
# 计算平均年龄
mean = titanic['age'].mean()
mean

In [ ]:
# 将缺失的年龄都用平均年龄来替代
titanic['age'].fillna(mean, inplace=True)
titanic.fillna(0, inplace=True)

- 现在，所有丢失的值都得到处理

### 20.2.6 对 `Titanic` 数据进行训练、测试...

#### (1) 数据拆分

In [ ]:
from sklearn.model_selection import train_test_split

titanic_target = titanic['survived']
titanic_data = titanic.drop(['name', 'row.names', 'survived'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(titanic_data, titanic_target, test_size=0.25, random_state=33)

In [ ]:
titanic_data.shape, X_train.shape

#### (2) 采用决策树


- 训练
- 测试
- 评价 —— 得分 = 0.83

In [ ]:
from sklearn import tree

dt = tree.DecisionTreeClassifier(criterion='entropy')
dt = dt.fit(X_train, y_train)

In [ ]:
from sklearn import metrics

y_pred = dt.predict(X_test)
print ("精度得分:{0:.3f}".format(metrics.accuracy_score(y_test, y_pred)), "\n")

## 20.3 特征选择 (Feature Selection)


- 注意到 —— 前述决策树算法，用了全部的特征

### 20.3.1 问题 —— 应该使用多少特征？


- 两种倾向
    - 使用全部信息，当然是多多益善
    - 限制 算法使用的特征数量


- 限制特征数量的原因
    - 原因之一 —— 关注相关性
        - 某些算法（包括决策树），无关的特征可能碰巧产生与目标的相关性，这未必正确，但易导致过拟合
        - 有些特征有可能高度相关，可能引起算法上的冗余
    - 原因之二 —— 考虑计算量
        - 过多的特征易引起大计算量，尤其是对于大数据问题
        - 与维度魔咒类似的问题，难于分析如下问题
        $$过多特征 + 过多样本$$

### 20.3.2 特征选择 —— 通过算法寻找最佳特征

#### (1) `Titanic` 特征如何变得臃肿？

- 增加到 581 个

In [ ]:
titanic.columns

#### (2) 要紧吗？


- 或许这里还不至于引起问题
- 但必须尽量避免 —— 可能出现的特征数急剧膨胀

#### (3) 决策树算法 —— 要避免过拟合


- 当分支基于很小的样本数时，对于未知数据的预测能力可能显著降低
- 决策树的解决办法
    -  调参：最大层数、叶结点的最小样本数

#### (4) 本节的解决办法


- 特征选择
    - 限制特征数 —— 如何限制？
    - 选择最有效的那些特征 —— 如何选择？

In [ ]:
from sklearn import feature_selection

#### (5) 特征选择函数

- 语法

```python
>>> feature_selection.SelectPercentile(score_func=<function f_classif>, percentile=10)
```

- 功能 —— 根据最高得分的分位数选择特征


- 参数
    - `score_func` 可调用函数，两个参数 `X` 和 `y`，返回 得分数组 `(scores, pvalues)`
    - `percentile` 拟保留的特征百分比，缺省值 10


- 属性
    - `scores_` 特征得分
    - `pvalues_` 特征得分 `p` 值

In [ ]:
fs = feature_selection.SelectPercentile(feature_selection.chi2, percentile=20)
X_train_fs = fs.fit_transform(X_train, y_train)

In [ ]:
feature_selection.chi2?

In [ ]:
dir(fs)

In [ ]:
# 查看特征的得分
fs.scores_, fs.scores_.shape

In [ ]:
X_train.shape, X_train_fs.shape

- 选取了 115 个特征

In [ ]:
X_train_fs

In [ ]:
dt.fit(X_train_fs, y_train) # 训练特征树

In [ ]:
# 测试

X_test_fs = fs.transform(X_test)   # 选取特征
y_pred_fs = dt.predict(X_test_fs)  # 预测所选取的特征

In [ ]:
# 评估略高于前
print ("精度得分:{0:.3f}".format(metrics.accuracy_score(y_test, y_pred_fs)),"\n")

#### (6) 交叉验证

In [ ]:
from sklearn import model_selection

percentiles = range(1, 100, 5) # [1, 6, 11, 16, 21, ...... ]
results = []

In [ ]:
for percentile in percentiles:
    fs = feature_selection.SelectPercentile(feature_selection.chi2, percentile=percentile)
    X_train_fs = fs.fit_transform(X_train, y_train)
    scores = model_selection.cross_val_score(dt, X_train_fs, y_train, cv=5)

    results = np.append(results, scores.mean())

In [ ]:
for p, r in zip(percentiles,results):
    print("{0:2},  {1}".format(p, r))

In [ ]:
optimal_percentil = np.where(results == results.max())[0]
#print ("Optimal percentil:{0}".format(percentiles[optimal_percentil]), "\n")
print ("最优百分位数:{0}".format(percentiles[optimal_percentil[0]]), "\n")

#### (7) 特征数 - 交叉验证评分 图


- 特征百分位数取 6 时，评分最高 0.879 左右
- 特征百分位数取 6 以上时，稳定评分在 0.86 左右

In [ ]:
plt.figure()

plt.xlabel("Percentile selected")
plt.ylabel("Cross validation accuracy)")
plt.plot(np.array(percentiles), results)

print ("平均得分:\n",results);

#### (8) 最佳表现 —— 相比最初 0.83 左右的得分，有轻微的改善

- 分位数 = 6
- 特征数 = 34
- 评分 = 0.857

In [ ]:
optimal_percentil = 6

In [ ]:
fs = feature_selection.SelectPercentile(feature_selection.chi2, percentile=optimal_percentil)

X_train_fs = fs.fit_transform(X_train, y_train)  # 特征选取
dt.fit(X_train_fs, y_train)

In [ ]:
X_test_fs = fs.transform(X_test)

In [ ]:
X_train_fs.shape, X_test_fs.shape

In [ ]:
y_pred_fs = dt.predict(X_test_fs)

print ("精度得分:{0:.3f}".format(metrics.accuracy_score(y_test, y_pred_fs)), "\n")

### 20.3.3 进一步改善


- 调整参数   `criterion` 选择不同的决策树准则

#### (1) 采用熵准则 —— `criterion='entropy'`

In [ ]:
dt = tree.DecisionTreeClassifier(criterion='entropy')
scores = model_selection.cross_val_score(dt, X_train_fs, y_train, cv=5)
print ("Entropy criterion accuracy on cv: {0:.3f}".format(scores.mean()))

#### (2) 采用基尼准则 —— `criterion='gini'`

In [ ]:
dt = tree.DecisionTreeClassifier(criterion='gini')
scores = model_selection.cross_val_score(dt, X_train_fs, y_train, cv=5)
print ("Gini criterion accuracy on cv: {0:.3f}".format(scores.mean()))

#### (3) 作用于测试集

In [ ]:
dt.fit(X_train_fs, y_train)

X_test_fs = fs.transform(X_test)
y_pred_fs = dt.predict(X_test_fs)

print ("精度得分: {0:.3f}".format(metrics.accuracy_score(y_test, y_pred_fs)),"\n")

#### (4) 讨论


- 以前发生过
    - 测试集的表现不如交叉验证的表现
    - 这是正常现象

## 20.4 模型选择(Model Selection)


- 前 2 节的工作至关重要
    - 特征提取 —— 预处理数据
    - 特征选择 —— 选出最理想的特征组


- 本节讨论另一个重要步骤 —— 选择算法参数 亦称 超参数(`hyperparameters`)

### 20.4.1 回顾基于朴素贝叶斯方法的文本分类问题


- 20 新闻组
- TF-IDF
- 基于多项式的朴素贝叶斯算法 —— `Multinomial Naïve Bayes Algorithm`
    - `MultinomialNB` 对象
- 对信息短文进行分类

#### (1) `MultinomialNB` 对象的重要参数

- `alpha` —— 用来调节光滑性的参数
    - 取缺省值 1.0 时，得分 0.89
    - 改进到取 0.01 时，得分 0.92


- 显然，参数 `alpha` 对分类效果影响显著


- 问题 —— `alpha=0.01` 是最佳参数吗？

#### (2) 重新加载 `fetch_20newsgroups` 数据集

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
news = fetch_20newsgroups(subset='all')
n_samples = 3000

In [ ]:
X_train = news.data[:n_samples]
y_train = news.target[:n_samples]

#### (3) 导入相关类


- MultinomialNB
- Pipeline
- TfidfVectorizer

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

#### (4) 设置 `stop_words`

In [ ]:
def get_stop_words():
    result = set()
    for line in open('data/stopwords_en.txt', 'r').readlines():
        result.add(line.strip())
    return result

stop_words = get_stop_words()

#### (5) 创建分类器

In [ ]:
clf = Pipeline([
    ('vect', TfidfVectorizer(
        stop_words=stop_words,
        token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('nb', MultinomialNB(alpha=0.01)),
])

#### (6) 交叉验证、 K-折

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

In [ ]:
def evaluate_cross_validation(clf, X, y, K):
    cv = KFold(K, shuffle=True, random_state=0)
    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print (("Mean score: {0:.3f} (+/-{1:.3f})").format(np.mean(scores), sem(scores)))

In [ ]:
evaluate_cross_validation(clf, X_train, y_train, 3)

### 20.4.2 参数优化分析

#### (1) 分析参数序列 —— 得到最优结果

- 定义函数

```python
>>> calc_params(X, y, clf, param_values, param_name, K)
```


- 功能 —— 实现对参数序列的计算、绘效果图


- 参数说明 —— 略

In [ ]:
def calc_params(X, y, clf, param_values, param_name, K):
    train_scores = np.zeros(len(param_values))
    test_scores = np.zeros(len(param_values))

    for i, param_value in enumerate(param_values):
        print (param_name, ' = ', param_value)
        clf.set_params(**{param_name:param_value})
        k_train_scores = np.zeros(K)
        k_test_scores = np.zeros(K)
        #cv = KFold(n_samples, K, shuffle=True, random_state=0)
        cv = KFold(K, shuffle=True, random_state=0)

        #for j, (train, test) in enumerate(cv):
        for j, (train, test) in enumerate(cv.split(X)):
            clf.fit([X[k] for k in train], y[train])
            k_train_scores[j] = clf.score([X[k] for k in train], y[train])
            k_test_scores[j] = clf.score([X[k] for k in test],y[test])

        train_scores[i] = np.mean(k_train_scores)
        test_scores[i] = np.mean(k_test_scores)

    plt.semilogx(param_values, train_scores, alpha=0.4, lw=2, c='b', label="Train")
    plt.semilogx(param_values, test_scores, alpha=0.4, lw=2, c='g', label="Test")

    plt.xlabel("Alpha values")
    plt.ylabel("Mean cross-validation accuracy")
    
    plt.legend()

    return train_scores, test_scores

#### (2) 参数序列 `alphas`

In [ ]:
alphas = np.logspace(-7, 0, 8)
print (alphas)

#### (3) 调用函数、K-折

```python
>>> calc_params(X_train, y_train, clf, alphas, 'nb__alpha', 3)`
```

- 注意 $K=3$

In [ ]:
train_scores, test_scores = calc_params(X_train, y_train, clf, alphas, 'nb__alpha', 3)

#### (4) 分析


- 训练效果总是好于测试效果
- 最佳测试效果对应的参数取值 $\alpha \in (10^{-2}, 10^{-1})$


- 过拟合区间 $\alpha \lt 10^{-2}$
    - 训练精度高，但测试精度过低


- 欠拟合区间 $\alpha \gt 10^{-1}$
    - 训练精度过低

### 20.4.3 采用支持向量机分类器 `SVC`


- 另一参数进行优化
    - `gamma`
    

- 参数说明
    - `gamma` 是选择 `RBF` 函数作为 `kernel` 后，该函数自带的一个参数
    - 决定了数据映射到新的特征空间后的分布
        - `gamma` 越大，支持向量越少
        - `gamma` 值越小，支持向量越多
    - 支持向量的个数影响训练与预测的速度

#### (1) 创建分类器

In [ ]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vect', TfidfVectorizer(
        stop_words=stop_words,
        token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('svc', SVC()),
    ])

#### (2) 参数序列 `gammas`

In [ ]:
gammas = np.logspace(-2, 1, 4)
gammas

In [ ]:
train_scores, test_scores = calc_params(X_train, y_train, clf, gammas,'svc__gamma', 3)

In [ ]:
gammas, train_scores, test_scores

#### (3) 结果分析


- $\gamma \lt 1$ 时，欠拟合分析
- $\gamma \gt 1$ 时，过拟合分析


- $\gamma = 1$ 时，效果最优
    - 训练得分 0.999
    - 测试得分 0.746

#### (4) 进一步思考

- 创建 `SVC` 对象的语法

```python
>>> SVC(C=1.0, kernel='rbf', degree=3,
        gamma='auto', coef0=0.0, shrinking=True,
        probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False,
        max_iter=-1, decision_function_shape='ovr', random_state=None)
```

- `SVC` 对象还有其它参数
    - `C`
    - `kernel`
    - 等


- 如果考虑更多的参数组合
    - 将产生新的复杂问题
    
    
- 怎么办？

## 20.5 参数搜索


- 网格搜索
- 平行网格搜索（略）

### 20.5.1 子模块 `model_selection` 中导入 `GridSearchCV`

In [ ]:
from sklearn.model_selection  import GridSearchCV

### 20.5.2 参数组合域

In [ ]:
parameters = {
    'svc__gamma': np.logspace(-2, 1, 4),
    'svc__C': np.logspace(-1, 1, 3),
}

### 20.5.3 创建分类器

In [ ]:
clf = Pipeline([
    ('vect', TfidfVectorizer(
        stop_words=stop_words,
        token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('svc', SVC()),
])

### 20.5.4 创建 `GridSearchCV` 对象

In [ ]:
gs = GridSearchCV(clf, parameters, verbose=2, refit=False, cv=3)

### 20.5.5 训练、评价、K-折

In [ ]:
%time _ = gs.fit(X_train, y_train)
gs.best_params_, gs.best_score_

### 20.5.6 小结


- 网格搜索得到更好的参数组合
    - `C=10`
    - `gamma=0.10`


- 相应的测试得分
    - 得分 0.827


- 对比前面的分析 —— 仅调整参数 `gamma`(保持 `C` 为 1)
    - 得分 0.746
    

- 更一般地考虑
    - 调整更多的 `TfidfVectorizer` 参数
    - 更加复杂
    

- 时间因素
    - 上面的分析用时 4+ 分钟(可能有微小变化)
    - 如果考虑更多的参数呢？
    - 时间呈几何级数上升......

### 20.5.7 有待进一步思考的问题


- 参数优化该如何做？

### 第20讲 结束